#### 1478. Allocate Mailboxes

* https://leetcode.com/problems/allocate-mailboxes/description/

#### Bloomberg

In [ ]:
# TC - 0(k * n^2)
# SC - O(n^2 + k*n)
# Decent Explanation - https://www.youtube.com/watch?v=ECcJUqdumIo
# GPT Explanation - https://chatgpt.com/c/694f8d86-09cc-8320-8eb3-fa941350fc69

from typing import List
from functools import lru_cache

class Solution:
    def minDistance(self, houses: List[int], k: int) -> int:
        # Sort houses so distance calculations make sense
        houses.sort()
        n = len(houses)

        # --------------------------------------------------
        # Step 1: Precompute cost[i][j]
        # cost[i][j] = minimum distance if ONE mailbox
        # serves houses from index i to j (inclusive)
        # --------------------------------------------------
        cost = [[0] * n for _ in range(n)]

        for i in range(n):
            for j in range(i, n):
                # Median minimizes sum of absolute distances
                mid = (i + j) // 2

                # Sum distance of each house in [i..j]
                # from the median house
                cost[i][j] = sum(
                    abs(houses[t] - houses[mid])
                    for t in range(i, j + 1)
                )

        # --------------------------------------------------
        # Step 2: Dynamic Programming with memoization
        #
        # dp(m, i) = minimum cost to cover houses
        # from index i to end using m mailboxes
        # --------------------------------------------------

        @lru_cache(None)
        def dp(m, i):
            # If all houses are covered
            if i == n:
                # Valid only if no mailboxes left
                return 0 if m == 0 else float('inf')

            # If houses remain but no mailboxes left
            if m == 0:
                return float('inf')

            res = float('inf')

            # Try placing one mailbox covering houses i..j
            # and solve remaining houses with m-1 mailboxes
            for j in range(i, n):
                res = min(
                    res,
                    cost[i][j] + dp(m - 1, j + 1)
                )

            return res

        # Start with k mailboxes covering from index 0
        return dp(k, 0)
